In [1]:
from collections import defaultdict
import numpy as np
import pdb
import dill
import random
import itertools
import matplotlib.pyplot as plt
from math import sqrt
import pandas as pd
from sklearn.metrics import f1_score, confusion_matrix, silhouette_score, davies_bouldin_score, mean_squared_error

In [2]:
class Lorentz:
    def __init__(self, s = 10, r = 28, b = 8/3):
        self.s = s
        self.r = r
        self.b = b

    def X(self, x, y, s):
        return s * (y - x)

    def Y(self, x, y, z, r):
        return (-x) * z + r * x - y

    def Z(self, x, y, z, b):
        return x * y - b * z

    def RK4(self, x, y, z, s, r, b, dt):
        k_1 = self.X(x, y, s)
        l_1 = self.Y(x, y, z, r)
        m_1 = self.Z(x, y, z, b)

        k_2 = self.X((x + k_1 * dt * 0.5), (y + l_1 * dt * 0.5), s)
        l_2 = self.Y((x + k_1 * dt * 0.5), (y + l_1 * dt * 0.5), (z + m_1 * dt * 0.5), r)
        m_2 = self.Z((x + k_1 * dt * 0.5), (y + l_1 * dt * 0.5), (z + m_1 * dt * 0.5), b)

        k_3 = self.X((x + k_2 * dt * 0.5), (y + l_2 * dt * 0.5), s)
        l_3 = self.Y((x + k_2 * dt * 0.5), (y + l_2 * dt * 0.5), (z + m_2 * dt * 0.5), r)
        m_3 = self.Z((x + k_2 * dt * 0.5), (y + l_2 * dt * 0.5), (z + m_2 * dt * 0.5), b)

        k_4 = self.X((x + k_3 * dt), (y + l_3 * dt), s)
        l_4 = self.Y((x + k_3 * dt), (y + l_3 * dt), (z + m_3 * dt), r)
        m_4 = self.Z((x + k_3 * dt), (y + l_3 * dt), (z + m_3 * dt), b)

        x += (k_1 + 2 * k_2 + 2 * k_3 + k_4) * dt * (1/6)
        y += (l_1 + 2 * l_2 + 2 * l_3 + l_4) * dt * (1/6)
        z += (m_1 + 2 * m_2 + 2 * m_3 + m_4) * dt * (1/6)

        return (x, y, z)

    def generate(self, dt, steps):
        x_0, y_0, z_0 = 1, 1, 1

        x_list = [x_0]
        y_list = [y_0]
        z_list = [z_0]

        i = 0

        while i < steps:
            x = x_list[i]
            y = y_list[i]
            z = z_list[i]

            position = self.RK4(x, y, z, self.s, self.r, self.b, dt)

            x_list.append(position[0])
            y_list.append(position[1])
            z_list.append(position[2])

            i += 1

        x_array = np.array(x_list)
        y_array = np.array(y_list)
        z_array = np.array(z_list)

        return x_array, y_array, z_array

In [3]:
xs, _, _ = Lorentz().generate(0.1, 100)
xs = (xs - xs.min()) / (xs.max() - xs.min())

In [4]:
def create_pat(xs, size_of_pattern, lenght_of_pattern):
    list_of_index = []
    for i in range(len(xs)):
        list_of_index.append(i)
    
    list_of_tuples_with_index = list(itertools.combinations(list_of_index, size_of_pattern))
    list_with_values = []
    for i in list_of_tuples_with_index:
        current_tuple = [False] * (i[-1] - i[0] + 1)
        for j in i:
            current_tuple[j - i[0]] = xs[j]
        if (len(current_tuple) <= lenght_of_pattern):
            list_with_values.append(current_tuple)
    return list_with_values

In [5]:
%%time

q = create_pat(xs, 5, 10)

CPU times: user 1min 56s, sys: 14.8 s, total: 2min 11s
Wall time: 2min 13s


In [6]:
epsilon = 0.05

In [7]:
len(q)

11718

In [8]:
def make_predict(p, q):
    lists_with_list_with_predicted_values = [0] * int(len(p))
    for i in range(len(lists_with_list_with_predicted_values)):
        lists_with_list_with_predicted_values[i] = []

    for i in range(len(p)):
        for j in q:
            if len(j) <= (len(p) - i):
                counter = 0
                for k in range(len(j)):
                    if j[k]:
                        if abs(p[i + k] - j[k]) <= epsilon:
                            counter += 1
                if (counter == len(j)):
                    for k in range(len(j)):
                        lists_with_list_with_predicted_values[i + k].append(j[k])
                        print(j[k], "добавлено число в элемент", i + k)
    return lists_with_list_with_predicted_values

In [9]:
predicted = make_predict(xs, q)
predicted

0.43661864602799916 добавлено число в элемент 0
0.48106163958052023 добавлено число в элемент 1
0.6357489179039438 добавлено число в элемент 2
1.0 добавлено число в элемент 3
0.8578413444179409 добавлено число в элемент 4
0.48106163958052023 добавлено число в элемент 1
0.6357489179039438 добавлено число в элемент 2
1.0 добавлено число в элемент 3
0.8578413444179409 добавлено число в элемент 4
0.4348845525448773 добавлено число в элемент 5
0.6357489179039438 добавлено число в элемент 2
1.0 добавлено число в элемент 3
0.8578413444179409 добавлено число в элемент 4
0.4348845525448773 добавлено число в элемент 5
0.25591465703200905 добавлено число в элемент 6
1.0 добавлено число в элемент 3
0.8578413444179409 добавлено число в элемент 4
0.4348845525448773 добавлено число в элемент 5
0.25591465703200905 добавлено число в элемент 6
0.1796224951372406 добавлено число в элемент 7
0.8578413444179409 добавлено число в элемент 4
0.4348845525448773 добавлено число в элемент 5
0.25591465703200905 д

0.1594685054847227 добавлено число в элемент 12
0.1802378746943853 добавлено число в элемент 13
0.06522149216998178 добавлено число в элемент 9
0.01412446727751653 добавлено число в элемент 10
0.05615289534687599 добавлено число в элемент 11
0.14019083115696687 добавлено число в элемент 12
0.182116837010264 добавлено число в элемент 13
0.01412446727751653 добавлено число в элемент 9
0.05615289534687599 добавлено число в элемент 10
0.14019083115696687 добавлено число в элемент 11
0.182116837010264 добавлено число в элемент 12
0.1618459994420985 добавлено число в элемент 13
0.02097098026226114 добавлено число в элемент 9
0.03144792365357535 добавлено число в элемент 10
0.11434479547087736 добавлено число в элемент 11
0.17684758061277697 добавлено число в элемент 12
0.17675430139643197 добавлено число в элемент 13
0.0390757023663827 добавлено число в элемент 9
0.014512163987326486 добавлено число в элемент 10
0.0839093020567306 добавлено число в элемент 11
0.16361265062432537 добавлено чи

0.17675430139643197 добавлено число в элемент 13
0.12051829829266514 добавлено число в элемент 14
0.0390757023663827 добавлено число в элемент 15
0.014512163987326486 добавлено число в элемент 16
0.16361265062432537 добавлено число в элемент 12
0.1856266792613335 добавлено число в элемент 13
0.14630674376076233 добавлено число в элемент 14
0.06555770566069558 добавлено число в элемент 15
0.009397294781438985 добавлено число в элемент 16
0.14202790477628024 добавлено число в элемент 12
0.18737699341674818 добавлено число в элемент 13
0.16782212421936493 добавлено число в элемент 14
0.09627220859466631 добавлено число в элемент 15
0.018331846388815128 добавлено число в элемент 16
0.18737699341674818 добавлено число в элемент 12
0.16782212421936493 добавлено число в элемент 13
0.09627220859466631 добавлено число в элемент 14
0.018331846388815128 добавлено число в элемент 15
0.02540444237711299 добавлено число в элемент 16
0.18082548600415574 добавлено число в элемент 12
0.1835057632149324

0.0332337144595181 добавлено число в элемент 16
0.034741767493607426 добавлено число в элемент 17
0.10572866650166175 добавлено число в элемент 18
0.16544320381597807 добавлено число в элемент 19
0.16917074736702362 добавлено число в элемент 20
0.034741767493607426 добавлено число в элемент 16
0.10572866650166175 добавлено число в элемент 17
0.16544320381597807 добавлено число в элемент 18
0.16917074736702362 добавлено число в элемент 19
0.12010981741407506 добавлено число в элемент 20
0.022133516452006976 добавлено число в элемент 16
0.08068196263760263 добавлено число в элемент 17
0.15333014285318977 добавлено число в элемент 18
0.17577469339673008 добавлено число в элемент 19
0.14103521295377708 добавлено число в элемент 20
0.018839127189645328 добавлено число в элемент 16
0.055724005455284144 добавлено число в элемент 17
0.13516136690432826 добавлено число в элемент 18
0.1768426350063366 добавлено число в элемент 19
0.15876841109219694 добавлено число в элемент 20
0.025577028771132

0.12010981741407506 добавлено число в элемент 20
0.047236108797284605 добавлено число в элемент 21
0.022133516452006976 добавлено число в элемент 22
0.08068196263760263 добавлено число в элемент 23
0.15333014285318977 добавлено число в элемент 24
0.14103521295377708 добавлено число в элемент 20
0.06871914221847295 добавлено число в элемент 21
0.018839127189645328 добавлено число в элемент 22
0.055724005455284144 добавлено число в элемент 23
0.13516136690432826 добавлено число в элемент 24
0.15876841109219694 добавлено число в элемент 20
0.09339598488548828 добавлено число в элемент 21
0.02557702877113203 добавлено число в элемент 22
0.033996736197137105 добавлено число в элемент 23
0.11153035083424628 добавлено число в элемент 24
0.1185864047766757 добавлено число в элемент 20
0.041800708252524715 добавлено число в элемент 21
0.019067585441226442 добавлено число в элемент 22
0.08418452710692943 добавлено число в элемент 23
0.1594685054847227 добавлено число в элемент 24
0.1420014987168

...

0.16492198785592493 добавлено число в элемент 81
0.19209853646206151 добавлено число в элемент 82
0.15495214610326413 добавлено число в элемент 83
0.07140830159864874 добавлено число в элемент 84
0.005211564740019722 добавлено число в элемент 85
0.13918619858895334 добавлено число в элемент 81
0.19226481858819344 добавлено число в элемент 82
0.17730720144049084 добавлено число в элемент 83
0.10616465157477732 добавлено число в элемент 84
0.019365904303170447 добавлено число в элемент 85
0.10463805559993616 добавлено число в элемент 81
0.18249824463689027 добавлено число в элемент 82
0.19251929031092835 добавлено число в элемент 83
0.13977389903350018 добавлено число в элемент 84
0.04802882096552335 добавлено число в элемент 85
0.1615018825189805 добавлено число в элемент 81
0.19912749597824592 добавлено число в элемент 82
0.16864903738234097 добавлено число в элемент 83
0.08517723451112448 добавлено число в элемент 84
0.003990443772717022 добавлено число в элемент 85
0.1654432038159780

0.047236108797284605 добавлено число в элемент 85
0.022133516452006976 добавлено число в элемент 86
0.08068196263760263 добавлено число в элемент 87
0.15333014285318977 добавлено число в элемент 88
0.17577469339673008 добавлено число в элемент 89
0.06871914221847295 добавлено число в элемент 85
0.018839127189645328 добавлено число в элемент 86
0.055724005455284144 добавлено число в элемент 87
0.13516136690432826 добавлено число в элемент 88
0.1768426350063366 добавлено число в элемент 89
0.018839127189645328 добавлено число в элемент 85
0.055724005455284144 добавлено число в элемент 86
0.13516136690432826 добавлено число в элемент 87
0.1768426350063366 добавлено число в элемент 88
0.15876841109219694 добавлено число в элемент 89
0.02557702877113203 добавлено число в элемент 85
0.033996736197137105 добавлено число в элемент 86
0.11153035083424628 добавлено число в элемент 87
0.17160998954968318 добавлено число в элемент 88
0.17215168972464137 добавлено число в элемент 89
0.0418007082525

0.05258830188576778 добавлено число в элемент 93
0.18737699341674818 добавлено число в элемент 89
0.16782212421936493 добавлено число в элемент 90
0.09627220859466631 добавлено число в элемент 91
0.018331846388815128 добавлено число в элемент 92
0.02540444237711299 добавлено число в элемент 93
0.16782212421936493 добавлено число в элемент 89
0.09627220859466631 добавлено число в элемент 90
0.018331846388815128 добавлено число в элемент 91
0.02540444237711299 добавлено число в элемент 92
0.11272345141523603 добавлено число в элемент 93
0.1835057632149324 добавлено число в элемент 89
0.12715227583322927 добавлено число в элемент 90
0.040345954078092175 добавлено число в элемент 91
0.00795083785121936 добавлено число в элемент 92
0.07826294146363928 добавлено число в элемент 93
0.19209853646206151 добавлено число в элемент 89
0.15495214610326413 добавлено число в элемент 90
0.07140830159864874 добавлено число в элемент 91
0.005211564740019722 добавлено число в элемент 92
0.043538013653987

0.10463805559993616 добавлено число в элемент 93
0.18249824463689027 добавлено число в элемент 94
0.19251929031092835 добавлено число в элемент 95
0.13977389903350018 добавлено число в элемент 96
0.04802882096552335 добавлено число в элемент 97
0.06517222402385574 добавлено число в элемент 93
0.1615018825189805 добавлено число в элемент 94
0.19912749597824592 добавлено число в элемент 95
0.16864903738234097 добавлено число в элемент 96
0.08517723451112448 добавлено число в элемент 97
0.16544320381597807 добавлено число в элемент 94
0.16917074736702362 добавлено число в элемент 95
0.12010981741407506 добавлено число в элемент 96
0.047236108797284605 добавлено число в элемент 97
0.022133516452006976 добавлено число в элемент 98
0.15333014285318977 добавлено число в элемент 94
0.17577469339673008 добавлено число в элемент 95
0.14103521295377708 добавлено число в элемент 96
0.06871914221847295 добавлено число в элемент 97
0.018839127189645328 добавлено число в элемент 98
0.1351613669043282

[[0.43661864602799916],
 [0.48106163958052023, 0.48106163958052023],
 [0.6357489179039438, 0.6357489179039438, 0.6357489179039438],
 [1.0, 1.0, 1.0, 1.0],
 [0.8578413444179409,
  0.8578413444179409,
  0.8578413444179409,
  0.8578413444179409,
  0.8578413444179409],
 [0.4348845525448773,
  0.4348845525448773,
  0.4348845525448773,
  0.4348845525448773,
  0.4348845525448773],
 [0.25591465703200905,
  0.25591465703200905,
  0.25591465703200905,
  0.25591465703200905,
  0.25591465703200905],
 [0.1796224951372406,
  0.1796224951372406,
  0.1796224951372406,
  0.1796224951372406,
  0.1796224951372406,
  0.16917074736702362,
  0.14103521295377708,
  0.15876841109219694,
  0.17215168972464137,
  0.1802378746943853,
  0.14200149871686646,
  0.1618459994420985,
  0.17675430139643197,
  0.14630674376076233,
  0.16782212421936493,
  0.1835057632149324,
  0.15495214610326413,
  0.17730720144049084,
  0.16864903738234097],
 [0.10319680292118334,
  0.10319680292118334,
  0.10319680292118334,
  0.1031

In [10]:
for i in range(len(predicted)):
    predicted[i] = np.average(predicted[i])

In [12]:
print(mean_squared_error(predicted, xs, squared=False))

0.006459600117041629


In [13]:

N = 5
WINDOW_MIN = 8
WINDOW = 15

In [31]:
import itertools
from sklearn.datasets import make_blobs
import numpy as np
import random

def generate_subsequences(window, num, short=False):
    values = list(range(window))
    result = []
    if short:
        for subseq in itertools.combinations(values, num):
            diff = WINDOW - WINDOW_MIN
            result.append(tuple([num + diff for num in subseq]))
    else:
        for subseq in itertools.combinations(values, num):
            result.append(subseq)
    
    return result

In [35]:
def generate_patterns():
    patterns1 = generate_subsequences(WINDOW_MIN - 1, N - 1, short=True)
    patterns2 = generate_subsequences(WINDOW - 1, N - 1)
    patterns3 = make_blobs.random.choices(patterns2[300:], k=35)    
    
    return patterns1 + patterns3

In [36]:
patterns = generate_patterns()

AttributeError: 'function' object has no attribute 'random'